In [ ]:
%run toricNecessitiesLessMemory.ipynb  #This is where the new Toric_Variety class lives
from treesGenerator import * #Loads in function/data needed for sampling the tree ensemble
# %run treesGenerator.ipynb  #If you want to run the Jupyter notebook version
import copy
from numpy import random

#These 2 files contain the ray/cone data for FRSTs of the two 3d reflexive polytopes that yield the largest number of tree ensemble geometries
from s1Data import *
from s2Data import *

#Build the initial Toric_Variety from the rays/cones in s1Data
polytopetoric = Toric_Variety(s1rays,s1cones)
# print('Initial h11: ', len(polytopetoric.edges())-3)

saveToFile = False  #Set true to save to file the timing dictionary defined below (h11s and time to calc. mori gens)
tvplot = []
h11p = []
#morigens = []
tips = []
#sample_list_cones = [1, 0, 2, 3, 5, 7, 10, 11, 13, 19, 20, 23, 25, 29, 30, 31, 35, 37, 39, 41, 43, 47, 49, 51, 53, 57, 59, 61, 64, 67, 71, 72, 67, 65, 63, 51]
#sample_list_edges = [0, 1, 2, 3, 5, 7, 10, 11, 13, 19, 20, 23, 25, 29, 30, 31, 35, 37, 39, 41, 43, 47, 49, 51, 53, 57, 59, 61, 64, 67, 71, 72, 100, 97, 89, 77]
if saveToFile:
    fyle1 = open("exampleCones.py", "w") #saves cones
    fyle2 = open("exampleRays.py", "w") #saves rays
    fyle1.write("conez = [")
    fyle2.write("raze = [")
for i in range(1, 37):
    #The max. number of face and edge blow ups for the s1Data set
    ncones = [2*i]
    nedges = [3*i]
    #ncones = [sample_list_cones[i]]
    #nedges = [sample_list_edges[i]]
    
    #Could instead uncomment this section to get more examples with various numbers of face/edge tree
    # ncones = [i+1 for i in range(len(polytopetoric.cones())) if ((i+1)%3==0 or i==1)]
    # nedges = []
    # for c in ncones:
    #     if c==72:
    #         nedges.append(107)
    #     else:
    #         nedges.append(c+10)
    print('ncones: ',ncones)
    print('nedges: ',nedges)
    timings = {}
    timings['h11s']=[]
    timings['mori']=[]
    num_torics = 0  #keep track of the number of new toric varieties made
    startTime = time.time()
    for blups in range(len(ncones)):  #iterate over each desired number of face trees to add
        stime = time.time()
        newtv = newrandomtoric(polytopetoric.edges(), polytopetoric.cones(), ncones[blups], nedges[blups])  #Add the trees
        newtv = Toric_Variety(*newtv)#Build the new toric variety
        tvplot.append(newtv)
        
        print('\nTime to add trees: ', round(time.time()-stime,5),'s')
        
        print('h11: ', len(newtv.edges())-3)# Get and record h11
        h11 = len(newtv.edges())-3
        h11p.append(h11)
        timings['h11s'].append(len(newtv.edges())-3)
        #print('cones: ',len(newtv.cones()))
        
        print('smooth? ',newtv.is_smooth())  #Check if the new toric variety is smooth
        
        stime = time.time()
        moriGens = newtv.mori_generators()  #Get the generators of the Mori cone and time it
        mori_cone = Cone(moriGens)
        kahler_cone = mori_cone.dual_cone()
        tip = kahler_cone.tip_of_stretched_cone(1)
        #morigens.append(moriGens)
        tips.append(tip)
        #print('mori cone: ', mori_cone)
        #print('totskc:', totskc)
        etime = time.time()
        print('Time to mori: ',round(etime-stime,5),'s')
        timings['mori'].append(etime-stime)
        
        if saveToFile:  #Save the results
            saverays = newtv.edges().tolist()
            savecones = newtv.cones()
            
            if blups==0:
                fyle1.write(str(savecones))
                fyle2.write(str(saverays))
            else:
                fyle1.write("," + str(savecones))
                fyle2.write("," + str(saverays))
        num_torics += 1
        
        if saveToFile:
            fyle1.write("]")
            fyle2.write("]")
            fyle1.close()
            fyle2.close()
            
        print("\nTotal number of toric varieties made: " + str(num_torics))
        print("Time taken to do everything: " + str(time.time() - startTime),'s')

/home/cytools/mounted_volume/full_lattice.py:23: UserWarning: AS: Check where these functions are being used and remove them accordingly!
  warnings.warn("AS: Check where these functions are being used and remove them accordingly!")


In [2]:
from collections import Counter, defaultdict

In [4]:
from itertools import combinations
from math import factorial

In [61]:
def compute_cy_volume(self, tloc):
        """
        **Description:**
        Computes the volume of the Calabi-Yau at a location in the Kähler cone.

        **Arguments:**
        - `tloc` *(array_like)*: A vector specifying a location in the Kähler
            cone.

        **Returns:**
        *(float)* The volume of the Calabi-Yau at the specified location.

        **Example:**
        We construct a Calabi-Yau hypersurface and find its volume at the tip
        of the stretched Kähler cone.
        ```python {5}
        p = Polytope([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],[-1,-1,-6,-9]])
        t = p.triangulate()
        cy = t.get_cy()
        tip = cy.toric_kahler_cone().tip_of_stretched_cone(1)
        cy.compute_cy_volume(tip)
        # 3.4999999988856496
        ```
        """
        intnums = self.intersection_numbers()
        xvol = 0
        #basis = self.divisor_basis()
        for ii in intnums:
            mult = np.prod([factorial(c)
                            for c in Counter(ii).values()])
            xvol += intnums[ii]*np.prod([tloc[int(j)] for j in ii])/mult
        return xvol

In [62]:
total_volume = []
for i in range(len(tvplot)):
    print(i, len(tvplot[i].edges()))
    v = compute_cy_volume(tvplot[i], tips[i])
    total_volume.append(v)

0 57
1 44
2 85
3 113
4 167
5 222
6 289
7 317
8 370
9 512
10 540
11 615
12 661
13 783
14 806
15 807
16 935
17 956
18 1023
19 1069
20 1108
21 1247
22 1264
23 1337
24 1404
25 1479
26 1547
27 1597
28 1659
29 1748
30 1831
31 1846
32 1943
33 1887
34 1827
35 1460


In [64]:
tvl = total_volume

In [65]:
whole_volume.extend(tvl)

In [67]:
nv = whole_volume

In [69]:
filename = 'totalvolume.py'
with open(filename, 'w') as file:
    file.write(f"whole_volume = {nv}\n")

In [1]:
def compute_divisor_volumes(self, tloc, in_basis=False):
        """
        **Description:**
        Computes the volume of the basis divisors at a location in the Kähler
        cone.

        The volume of the ith divisor is 0.5*kappa_{ijk} t^j t^k.

        **Arguments:**
        - `tloc` *(array_like)*: A vector specifying a location in the Kähler
            cone.
        - `in_basis` *(bool, optional, default=False)*: When set to True, the
            volumes of the current basis of divisors are computed. Otherwise,
            the volumes of all prime toric divisors are computed.

        
        **Example:**
        We construct a Calabi-Yau hypersurface and find the volumes of the
        prime toric divisors at the tip of the stretched Kähler cone.
        ```python {5}
        p = Polytope([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],[-1,-1,-6,-9]])
        t = p.triangulate()
        cy = t.get_cy()
        tip = cy.toric_kahler_cone().tip_of_stretched_cone(1)
        cy.compute_divisor_volumes(tip)
        # array([ 2.5       , 23.99999999, 16.        ,  2.5       ,  2.5       ,
        #         0.5       ])
        ```
        """
        intnums = self.intersection_numbers()
        #basis = self.divisor_basis()
        #if len(basis.shape) == 2: # If basis is matrix
        tau = np.zeros(len(self.edges()), dtype=float)
        for ii in intnums:
            c = Counter(ii)
            for j in c.keys():
                tau[j] += intnums[ii] * np.prod([tloc[k]**(c[k]-(j==k))/factorial(c[k]-(j==k)) for k in c.keys()])
        return np.array(tau)

In [28]:
divisor_v = []
for i in range(len(tvplot)):
    print(i, len(tvplot[i].edges()))
    divisor_list = compute_divisor_volumes(tvplot[i], tips[i]).tolist()
    divisor_v.append(divisor_list)

0 810
1 1290
2 1459
3 434
4 1974
5 1235
6 1147
7 1484
8 1413
9 1103
10 1028
11 1158
12 519
13 239
14 197
15 405
16 1221
17 1023
18 1467
19 1158
20 1193
21 1355
22 724
23 848
24 538
25 1363
26 593
27 1577
28 844
29 1812
30 691
31 864
32 1274
33 801
34 922
35 1435


In [39]:
filename = 'divisorvolume.py'
with open(filename, 'w') as file:
    file.write(f"divisor_volume = {divisor_volume}\n")

In [43]:
filename = 'h11test.py'
with open(filename, 'w') as file:
    file.write(f"h11plot = {h11p}\n")